# HuggingFace Inference Baseline

Training notebook: https://www.kaggle.com/thedrcat/feedback-prize-huggingface-baseline-training

In [1]:
# Config
batch_size = 1
min_tokens = 5
tok_checkpoint = './longformer-base-4096/'
model_checkpoint = './longformer-base-4096/pytorch_model.bin'

In [2]:
# Load data
import pandas as pd

train = pd.read_csv('../../input/feedback-prize-2021/train.csv')
train.head(1)

test = pd.read_csv('../../input/feedback-prize-2021/sample_submission.csv')
test.head(1)

,id,class,predictionstring
0,18409261F5C2,NaN,NaN


In [3]:
# Setup dictionaries
classes = train.discourse_type.unique().tolist()

from collections import defaultdict

tags = defaultdict()
for i, c in enumerate(classes):
    tags[f'B-{c}'] = i
    tags[f'I-{c}'] = i + len(classes)
tags[f'O'] = len(classes) * 2
tags[f'Special'] = -100
l2i = dict(tags)

i2l = defaultdict()
for k, v in l2i.items(): 
    i2l[v] = k
i2l[-100] = 'Special'
i2l = dict(i2l)

In [4]:
# Helper functions
from pathlib import Path

test_path = Path('../../input/feedback-prize-2021/test')

def get_test_text(ids):
    with open(test_path/f'{ids}.txt', 'r') as file: data = file.read()
    return data

In [5]:
# Tokenizer
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tok_checkpoint, add_prefix_space=True)

In [6]:
# Load model
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch

model = AutoModelForTokenClassification.from_pretrained(tok_checkpoint, num_labels=len(i2l)-1)

model.load_state_dict(torch.load(model_checkpoint))
model.eval();

In [7]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [8]:
# We'll use trainer with the loaded model to run inference on test set
trainer = Trainer(
    model,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [9]:
# code that will convert our predictions into prediction strings. we'll skip visualization here. 
# this most likely requires some refactoring

def get_class(c):
    if c == 14: return 'Other'
    else: return i2l[c][2:]

def pred2span(pred, example, viz=False, test=False):
    example_id = example['id']
    n_tokens = len(example['input_ids'])
    classes = []
    all_span = []
    for i, c in enumerate(pred.tolist()):
        if i == n_tokens-1:
            break
        if i == 0:
            cur_span = example['offset_mapping'][i]
            classes.append(get_class(c))
        elif i > 0 and (c == pred[i-1] or (c-7) == pred[i-1]):
            cur_span[1] = example['offset_mapping'][i][1]
        else:
            all_span.append(cur_span)
            cur_span = example['offset_mapping'][i]
            classes.append(get_class(c))
    all_span.append(cur_span)
    
    if test: text = get_test_text(example_id)
    else: text = get_raw_text(example_id)
        
    # map token ids to word (whitespace) token ids
    predstrings = []
    for span in all_span:
        span_start = span[0]
        span_end = span[1]
        before = text[:span_start]
        token_start = len(before.split())
        if len(before) == 0: token_start = 0
        elif before[-1] != ' ': token_start -= 1
        num_tkns = len(text[span_start:span_end+1].split())
        tkns = [str(x) for x in range(token_start, token_start+num_tkns)]
        predstring = ' '.join(tkns)
        predstrings.append(predstring)
                    
    rows = []
    for c, span, predstring in zip(classes, all_span, predstrings):
        e = {
            'id': example_id,
            'discourse_type': c,
            'predictionstring': predstring,
            'discourse_start': span[0],
            'discourse_end': span[1],
            'discourse': text[span[0]:span[1]+1]
        }
        rows.append(e)


    df = pd.DataFrame(rows)
    df['length'] = df['discourse'].apply(lambda t: len(t.split()))
    
    # short spans are likely to be false positives, we can choose a min number of tokens based on validation
    df = df[df.length > min_tokens].reset_index(drop=True)

    return df

In [10]:
# Load test data
import os 

files = os.listdir('../../input/feedback-prize-2021/test')
ids = [x.split('.')[0] for x in files]

df_test = pd.DataFrame()
df_test['id'] = ids
df_test['text'] = df_test['id'].apply(get_test_text)
df_test

,id,text
0,0FB0700DAF44,"During a group project, have you ever asked a ..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D46BCB48440A,"When people ask for advice,they sometimes talk..."
3,D72CB1C11673,Making choices in life can be very difficult. ...
4,DF920E0A7337,Have you ever asked more than one person for h...


In [11]:
from datasets import Dataset

test_ds = Dataset.from_pandas(df_test)
test_ds

Dataset({
    features: ['id', 'text'],
    num_rows: 5
})

In [12]:
def tokenize_for_test(examples):

    o = tokenizer(examples['text'], truncation=True, return_offsets_mapping=True, max_length=4096)
  
    return o

In [13]:
tokenized_test = test_ds.map(tokenize_for_test)
tokenized_test

0ex [00:00, ?ex/s]

Dataset({
    features: ['id', 'text', 'input_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 5
})

In [14]:
predictions, _, _ = trainer.predict(tokenized_test)

The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: id, offset_mapping, text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids are automatically padded from 1304 to 1536 to be a multiple of `config.attention_window`: 512


In [15]:
import numpy as np

preds = np.argmax(predictions, axis=-1)
predictions.shape, preds.shape

((5, 1304, 15), (5, 1304))

In [16]:
dfs = []
for i in range(len(tokenized_test)):
    dfs.append(pred2span(preds[i], tokenized_test[i], test=True))

pred_df = pd.concat(dfs, axis=0)
pred_df['class'] = pred_df['discourse_type']

In [17]:
sub = pred_df[['id', 'class', 'predictionstring']]

In [18]:
!mkdir ../../output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
sub.to_csv('../../output/submission.csv', index=False)

In [51]:
# !kaggle competitions submit -c "feedback-prize-2021" -f "../../output/submission.csv" -m "huggingface-longformer-baseline"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████████████████████████████████| 13.0k/13.0k [00:02<00:00, 5.72kB/s]
400 - Bad Request


In [60]:
# !kaggle datasets init -p "./longformer-base-4096"
# !kaggle datasets create -p "./longformer-base-4096"
!kaggle datasets version -p "./longformer-base-4096" -m "v0.0.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Starting upload for file tokenizer.json
100%|██████████████████████████████████████| 2.01M/2.01M [00:02<00:00, 1.04MB/s]
Upload successful: tokenizer.json (2MB)
Starting upload for file special_tokens_map.json
100%|████████████████████████████████████████████| 239/239 [00:02<00:00, 113B/s]
Upload successful: special_tokens_map.json (239B)
Starting upload for file pytorch_model.bin
100%|████████████████████████████████████████| 565M/565M [00:34<00:00, 17.3MB/s]
Upload successful: pytorch_model.bin (565MB)
Starting upload for file tokenizer_config.json
100%|████████████████████████████████████████████| 368/368 [00:01<00:00, 216B/s]
Upload successful: tokenizer_config.js

In [53]:
!kaggle competitions submissions "feedback-prize-2021"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
fileName                                        date                 description                                              status    publicScore  privateScore  
----------------------------------------------  -------------------  -------------------------------------------------------  --------  -----------  ------------  
Feedback Prize HuggingFace Baseline: Inference  2022-02-07 14:38:19  Notebook Feedback Prize HuggingFace Baseline: Inference  complete  0.610        None          
